In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
tf.random.set_seed(50)

import numpy as np
import os
import time

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
tf.__version__

'2.11.0'

In [6]:
#Reading the learning Data
text = open("shakespeare.txt", 'rb').read().decode(encoding='utf-8')

#preprocessing
text=text.lower()

In [7]:
#Sample Text Data
print(text[:250])

the sonnets

by william shakespeare

from fairest creatures we desire increase,
that thereby beauty's rose might never die,
but as the riper should by time decease,
his tender heir might bear his memory:
but thou contracted to thine own bright eyes,



In [8]:
#Vocabulary
vocab = sorted(set(text))

In [9]:
get_id_from_char = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [10]:
get_char_from_id = tf.keras.layers.StringLookup(vocabulary=get_id_from_char.get_vocabulary(), invert=True, mask_token=None)

In [11]:
def generate_text(ids):
  return tf.strings.reduce_join(get_char_from_id(ids), axis=-1)

def lambda_split(sequence):
    return sequence[:-1], sequence[1:]

In [12]:
text_ids = get_id_from_char(tf.strings.unicode_split(text, 'UTF-8'))
print(text_ids)
print(len(text))

tf.Tensor([32 20 17 ... 17 26 16], shape=(93677,), dtype=int64)
93677


In [13]:

dataset = tf.data.Dataset.from_tensor_slices(text_ids)
stride = 3
window = 40
## creating dataset with window size and stride
dataset = dataset.window(size=window+1,shift=stride, drop_remainder=True)

## Function to flat the dataset from the datasets
def flat_function(batch):
    return batch.batch(window+1, drop_remainder=True)

## Use above function to apply on the loaded dataset
dataset_sequence = dataset.flat_map(flat_function)

count = 0
for x in dataset_sequence:
  count+=1
print("Number of training Samples = ",count)
print(f'Vocab Size {len(vocab)} ')

Number of training Samples =  31213
Vocab Size 38 


In [14]:
C = tf.constant(len(vocab))

#Function to Generate the input and target sequences and perform one hot encoding
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    input_text = tf.one_hot(
        input_text, C, on_value = 1.0, off_value = 0.0, axis =-1)
    target_text = tf.one_hot(
        target_text, C, on_value = 1.0, off_value = 0.0, axis =-1)
    
    return input_text, target_text

In [15]:
dataset = dataset_sequence.map(split_input_target)

In [16]:
#
for sample_input, sample_target in dataset.take(1):
    ids = tf.argmax(sample_input, axis=1)
    print("Input :", generate_text(ids).numpy())
    print("\nOne Hot encoding representation for above text",sample_input)
    ids = tf.argmax(sample_target, axis=1)
    print("\n\nTarget:", generate_text(ids).numpy())
    print("\nOne Hot encoding representation for above text",sample_target)

Input : b'the sonnets\n\nby william shakespeare\n\nfro'

One Hot encoding representation for above text tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(40, 38), dtype=float32)


Target: b'he sonnets\n\nby william shakespeare\n\nfrom'

One Hot encoding representation for above text tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(40, 38), dtype=float32)


In [17]:
batch = 32
buffer = 10000
dataset = dataset_sequence.map(lambda_split)
dataset = (dataset.shuffle(buffer).batch(batch, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))


In [18]:
# Vocabulary Size for embedding layer
vocabulary_size = len(get_id_from_char.get_vocabulary())

# dimension for embedding layer
embedding_dimensions = 128

# LSTM units
LSTM_units = 256

In [19]:
class LSTMModel(tf.keras.Model):
  def __init__(self, vocabulary_size, embedding_dimension, LSTM_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocabulary_size, embedding_dimension)
    self.lstm = tf.keras.layers.LSTM(LSTM_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocabulary_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.lstm.get_initial_state(x)
    x= self.lstm(x, initial_state=states, training=training)
    states=x[1:]
    x=x[0]
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [21]:
#Creating LSTMMODEL Object
model = LSTMModel(vocabulary_size=vocabulary_size,embedding_dimension=embedding_dimensions,LSTM_units=LSTM_units)

In [22]:
for sample_input_batch, sample_target_batch in dataset.take(1):
    sample_batch_predictions = model(sample_input_batch)
    print(sample_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(32, 40, 39) # (batch_size, sequence_length, vocab_size)


In [23]:
model.summary()

Model: "lstm_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  4992      
                                                                 
 lstm (LSTM)                 multiple                  394240    
                                                                 
 dense (Dense)               multiple                  10023     
                                                                 
Total params: 409,255
Trainable params: 409,255
Non-trainable params: 0
_________________________________________________________________


In [24]:
indexes = tf.random.categorical(sample_batch_predictions[0], num_samples=1)
indexes = tf.squeeze(indexes, axis=-1).numpy()

In [25]:
print("Input:\n", generate_text(sample_input_batch[0]).numpy())
print()
print("Next Char Predictions:\n", generate_text(indexes).numpy())

Input:
 b'om nature hath not made for store,\nharsh'

Next Char Predictions:
 b'iy.!fdz;:ot  dpbou[UNK]hdv:cgiucr(:i)mfqkiyw'


In [26]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [27]:
mean_loss = loss(sample_target_batch, sample_batch_predictions)
print("Prediction shape: ", sample_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (32, 40, 39)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.6637626, shape=(), dtype=float32)


In [28]:
tf.exp(mean_loss).numpy()

39.007835

In [29]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=loss)

In [30]:
epochs = 1000

In [29]:
history = model.fit(dataset, epochs=epochs)

Epoch 1/1000
975/975 [==============================] - 14s 11ms/step - loss: 2.0492
Epoch 2/1000
975/975 [==============================] - 9s 8ms/step - loss: 1.5475
Epoch 3/1000
975/975 [==============================] - 9s 8ms/step - loss: 1.3537
Epoch 4/1000
975/975 [==============================] - 10s 9ms/step - loss: 1.1996
Epoch 5/1000
975/975 [==============================] - 11s 10ms/step - loss: 1.0597
Epoch 6/1000
975/975 [==============================] - 9s 8ms/step - loss: 0.9355
Epoch 7/1000
975/975 [==============================] - 8s 7ms/step - loss: 0.8334
Epoch 8/1000
975/975 [==============================] - 9s 8ms/step - loss: 0.7502
Epoch 9/1000
975/975 [==============================] - 9s 8ms/step - loss: 0.6853
Epoch 10/1000
975/975 [==============================] - 9s 8ms/step - loss: 0.6334
Epoch 11/1000
975/975 [==============================] - 8s 7ms/step - loss: 0.5924
Epoch 12/1000
975/975 [==============================] - 9s 8ms/step - loss: 0.5

In [30]:
class Char_generator(tf.keras.Model):
  def __init__(self, model, get_char_from_id, get_id_from_char):
    super().__init__()
    self.model = model
    self.get_char_from_id = get_char_from_id
    self.get_id_from_char = get_id_from_char

    # logic to not print [unk]
    skip_ids = self.get_id_from_char(['[UNK]'])[:, None]
    mask = tf.SparseTensor(values=[-float('inf')]*len(skip_ids),indices=skip_ids,dense_shape=[len(get_id_from_char.get_vocabulary())])
    self.mask = tf.sparse.to_dense(mask)

  @tf.function
  def predict_next_char(self, inputs, states=None):
    # Convert strings to token IDs.
    string_input = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.get_id_from_char(string_input).to_tensor()

    # predict the output for given input string
    output, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last char.
    output = output[:, -1, :]
    
    output = output + self.mask

    output_ids = tf.random.categorical(output, num_samples=1)
    output_ids = tf.squeeze(output_ids, axis=-1)

    # convert to string from ids
    output_string = self.get_char_from_id(output_ids)

    # Return the characters and model state.
    return output_string, states

In [31]:
char_gen_model = Char_generator(model, get_char_from_id, get_id_from_char)

In [81]:
states = None
next_char = tf.constant(['inspired by you'])
result = [next_char]

for n in range(400):
  next_char, states = char_gen_model.predict_next_char(next_char, states=states)
  result.append(next_char)
  next_char = tf.constant(tf.strings.join(result))

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

inspired by youth,
and sweets grown common lose their dear victories once foiled,
is from the book of honour razed quite,
and all the rest forgot for which he toiled:
then happy i that love and am beloved
where i may not remove nor be removed.
  
lord of my love, to whom in vew shore away,
death's second self that seals up all in rest.  
o surn dear head, else true, love twinness shaken
as i by yours, y'have passed a hell of time,
and i a tyrant have no leisure taken
to weigh how once i suffered in your crime.
o that our night of woe might have remembered
my deepest sense, how hard true sorrow hits,
and soon to you, as you to medic women's spring: forth where,
and consument that your love taught it this alchemy?
to make of monsters, and things indigest,
such cherubins as your sweet self resembote,
but that sorrt thou lead away,
if thou wouldst almoward of a conquered woe,
give not a windy night a woman coloured ill.
to win thy presence is gracious and kind,
or to thy self at least kind

In [33]:
from google.colab import files


# Specify export directory and use tensorflow to save your_model
export_dir = './saved_model'
tf.saved_model.save(model, export_dir=export_dir)

In [37]:
print(help(history))

Help on History in module keras.callbacks object:

class History(Callback)
 |  Callback that records events into a `History` object.
 |  
 |  This callback is automatically applied to
 |  every Keras model. The `History` object
 |  gets returned by the `fit` method of models.
 |  
 |  Example:
 |  
 |  >>> model = tf.keras.models.Sequential([tf.keras.layers.Dense(10)])
 |  >>> model.compile(tf.keras.optimizers.SGD(), loss='mse')
 |  >>> history = model.fit(np.arange(100).reshape(5, 20), np.zeros(5),
 |  ...                     epochs=10, verbose=1)
 |  >>> print(history.params)
 |  {'verbose': 1, 'epochs': 10, 'steps': 1}
 |  >>> # check the keys of history object
 |  >>> print(history.history.keys())
 |  dict_keys(['loss'])
 |  
 |  Method resolution order:
 |      History
 |      Callback
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  on_epoch_end(self, epoch, logs=None)
 |

In [74]:
hold_outtext = open("hold-out.txt", 'rb').read().decode(encoding='utf-8')
hold_outtext=hold_outtext.lower()
print(hold_outtext)

the little love-god lying once asleep,
laid by his side his heart-inflaming brand,
whilst many nymphs that vowed chaste life to keep,
came tripping by, but in her maiden hand,
the fairest votary took up that fire,
which many legions of true hearts had warmed,
and so the general of hot desire,
was sleeping by a virgin hand disarmed.
this brand she quenched in a cool well by,
which from love's fire took heat perpetual,
growing a bath and healthful remedy,
for men discased, but i my mistress' thrall,
came there for cure and this by that i prove,  
love's fire heats water, water cools not love.



In [75]:
text_ids = get_id_from_char(tf.strings.unicode_split(hold_outtext, 'UTF-8'))
print(text_ids)
print(len(hold_outtext))
dataset = tf.data.Dataset.from_tensor_slices(text_ids)

## Create dataset of datasets with a specific window and shift size
dataset = dataset.window(size=window+1,shift=stride, drop_remainder=True)

## Function to flat the dataset from the datasets
def flat_function(batch):
    return batch.batch(window+1, drop_remainder=True)

## Use above function to apply on the loaded dataset
dataset_sequence = dataset.flat_map(flat_function)

count = 0
for x in dataset_sequence:
  count+=1
print("Number of training Samples = ",count)
print(f'Vocab Size {len(vocab)} ')

dataset = dataset_sequence.map(split_input_target)
num_records = tf.data.experimental.cardinality(dataset)

# Print the number of records
print(num_records)

for sample_input, sample_target in dataset.take(1):
    ids = tf.argmax(sample_input, axis=1)
    print("Input :", generate_text(ids).numpy())
    print("\nOne Hot encoding representation for above text",sample_input)
    ids = tf.argmax(sample_target, axis=1)
    print("\n\nTarget:", generate_text(ids).numpy())
    print("\nOne Hot encoding representation for above text",sample_target)

validation_dataset = sequences.map(lambda_split)
validation_dataset = (validation_dataset.shuffle(buffer).batch(batch, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))


tf.Tensor(
[32 20 17  2 24 21 32 32 24 17  2 24 27 34 17  8 19 27 16  2 24 37 21 26
 19  2 27 26 15 17  2 13 31 24 17 17 28  7  0  1 24 13 21 16  2 14 37  2
 20 21 31  2 31 21 16 17  2 20 21 31  2 20 17 13 30 32  8 21 26 18 24 13
 25 21 26 19  2 14 30 13 26 16  7  0  1 35 20 21 24 31 32  2 25 13 26 37
  2 26 37 25 28 20 31  2 32 20 13 32  2 34 27 35 17 16  2 15 20 13 31 32
 17  2 24 21 18 17  2 32 27  2 23 17 17 28  7  0  1 15 13 25 17  2 32 30
 21 28 28 21 26 19  2 14 37  7  2 14 33 32  2 21 26  2 20 17 30  2 25 13
 21 16 17 26  2 20 13 26 16  7  0  1 32 20 17  2 18 13 21 30 17 31 32  2
 34 27 32 13 30 37  2 32 27 27 23  2 33 28  2 32 20 13 32  2 18 21 30 17
  7  0  1 35 20 21 15 20  2 25 13 26 37  2 24 17 19 21 27 26 31  2 27 18
  2 32 30 33 17  2 20 17 13 30 32 31  2 20 13 16  2 35 13 30 25 17 16  7
  0  1 13 26 16  2 31 27  2 32 20 17  2 19 17 26 17 30 13 24  2 27 18  2
 20 27 32  2 16 17 31 21 30 17  7  0  1 35 13 31  2 31 24 17 17 28 21 26
 19  2 14 37  2 13  2 34 21 30 19 21 26 

In [80]:
# model evaluation and print perplexity
test_loss = model.evaluate(validation_dataset, verbose=2)
print('Test Perplexity:', np.exp(test_loss))

5/5 - 0s - loss: 7.4239 - 92ms/epoch - 18ms/step
Test Perplexity: 1675.514115646483
